<font size="8"> **Adding environmental data from available observations to unique background points** </font>  
In this notebook, we will extract environmental data from sea ice and sea surface temperature observations using the coordinates of the reported crabeater sightings.

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/41925/status,
Dashboard: /proxy/41925/status,Workers: 4
Total threads: 12,Total memory: 48.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46373,Workers: 4
Dashboard: /proxy/41925/status,Total threads: 12
Started: Just now,Total memory: 48.00 GiB
Comm: tcp://127.0.0.1:44847,Total threads: 3
Dashboard: /proxy/34933/status,Memory: 12.00 GiB
Nanny: tcp://127.0.0.1:40135,


# Adding values for dynamic variables
Given the amount of crabeater seal observations and the time period covered by this dataset, the extraction of these values may take some time. It is recommended to save the data frame after every time a new variable is extracted. This way we can avoid losing data.

In [4]:
#Loading dataset as pandas data frame
crabeaters = pd.read_csv('../Biological_Data/BG_points/unique_background_5x_obs_grid.csv')

#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)

#Checking results
crabeaters

,date,year,month,longitude,latitude,xt_ocean,yt_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,1999-12,1999,12,73.85,-69.25,73.85,-69.239,summer,weaning,1990,East Indian,Antarctic,0,NaN,504.582,140.256,NaN
1,1987-11,1987,11,71.65,-69.15,71.65,-69.155,autumn,weaning,1980,Central Indian,Antarctic,0,NaN,504.302,156.205,NaN
2,1987-11,1987,11,79.05,-69.15,79.05,-69.155,autumn,weaning,1980,Central Indian,Antarctic,0,NaN,506.018,204.310,NaN
3,1997-12,1997,12,74.55,-69.05,74.55,-69.070,summer,weaning,1990,Central Indian,Antarctic,0,89.715,485.126,121.468,797.050
4,1999-12,1999,12,75.35,-69.05,75.35,-69.070,summer,weaning,1990,East Indian,Antarctic,0,89.702,485.826,122.080,729.212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10774,1987-11,1987,11,69.85,-60.75,69.85,-60.739,autumn,weaning,1980,Central Indian,Antarctic,0,89.961,408.352,544.668,4415.736
10775,1988-12,1988,12,112.95,-60.75,112.95,-60.739,summer,weaning,1980,Central Indian,Antarctic,0,89.727,228.142,294.787,4340.138
10776,1999-12,1999,12,72.85,-60.15,72.85,-60.147,summer,weaning,1990,Central Indian,Antarctic,0,89.973,458.773,632.560,3646.319
10777,1999-12,1999,12,71.05,-59.55,71.05,-59.544,summer,weaning,1990,East Indian,Antarctic,0,89.971,527.919,681.598,4711.166


## Loading environmental data from observations

In [159]:
#Creating dictionary with useful information
varDict = {'var_name': 'dist_ice_edge_km',
           #Folder containing obs
           'obs_main': '/g/data/v45/la6889/Chapter2_Crabeaters/SeaIceObs/Distance_Edge/*.nc',
           #Output folder
           'base_out': '../Environmental_Data/Env_obs/'}

In [160]:
#Getting list of all obs in folder
files_var = sorted(glob(varDict['obs_main']))

#Loading all data into single dataset
var_df = xr.open_mfdataset(files_var)
var_df = var_df.rename_vars({'dist_km': 'dist_ice_edge_km'})
# var_df = var_df.rename_vars({'__xarray_dataarray_variable__': 'SST_degC'})
var_df = var_df.dist_ice_edge_km.rename({'lon': 'xt_ocean', 'lat': 'yt_ocean'})

#Subsetting data to Indian sectors
var_df = var_df.sel(yt_ocean = slice(-80, -40), xt_ocean = slice(30, 170))

#Checking results
var_df

<xarray.DataArray 'dist_ice_edge_km' (time: 494, yt_ocean: 713, xt_ocean: 1400)>
dask.array<getitem, shape=(494, 713, 1400), dtype=float64, chunksize=(1, 713, 1400), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 1978-11-15 1978-12-15 ... 2019-12-15
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -45.18 -45.11 -45.04
  * xt_ocean  (xt_ocean) float64 30.05 30.15 30.25 30.35 ... 169.8 169.9 170.0
Attributes:
    units:      km
    long_name:  distance to nearest neighbour

## Extracting environmental data
We will use the `latitude` and `longitude` columns together with the `event_date` column from the crabeater seal observations to find the corresponding grid cell in the model outputs and extract the value of the environmental factor of our interest.

In [264]:
#Getting coordinates from the crabeater data
lat = xr.DataArray(crabeaters.latitude)
lon = xr.DataArray(crabeaters.longitude)
#Getting data of observation from the crabeater data
time = xr.DataArray(crabeaters.apply(lambda x: pd.to_datetime(f'{x.date}-16'), axis = 1))

## Extracting data

In [265]:
#Extracting data
var_sub = var_df.sel(time = time, yt_ocean = lat, xt_ocean = lon, method = 'nearest')

#Transforming to data frame
var_pd = var_sub.to_dataframe().sort_values(['time', 'xt_ocean', 'yt_ocean'])
#Adding year and month
var_pd['year'] = var_pd.time.dt.year
var_pd['month'] = var_pd.time.dt.month
#Removing time column that is no longer needed
var_pd.drop(columns = 'time', inplace = True)
#Finding name of columns to round up
round_cols = [i for i in var_pd.columns if 'ocean' in i]
#Rounding coordinate values prior to merging
var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
#Renaming variable to be added to crabeater data
var_pd.rename(columns = {varDict['var_mod']: varDict['var_short_name']}, inplace = True)
#Getting column names for merging
cols = var_pd.drop(columns = varDict['var_short_name']).columns.tolist()

#Checking results
print(cols); var_pd

['yt_ocean', 'xt_ocean', 'year', 'month']


,yt_ocean,xt_ocean,dist_ice_edge_km,year,month
dim_0,,,,,
9662,-63.762,76.05,325.720123,1981,12
213,-67.043,76.15,690.285745,1981,12
167,-67.170,76.45,705.065718,1981,12
3328,-64.846,76.95,449.988341,1981,12
1752,-65.142,78.85,502.974930,1981,12
...,...,...,...,...,...
8349,-64.070,135.95,176.057370,2013,12
6879,-64.244,136.95,206.443250,2013,12
3795,-64.846,141.55,335.567183,2013,12


## Joining masked data frame with background data frame
We will use the grid cell coordinates and dates to perform this join.

In [165]:
crabeaters = crabeaters.merge(var_pd, on = cols, how = 'left')
crabeaters = crabeaters.drop_duplicates()
crabeaters

,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,...,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m,SIC,SST_degC,lt_pack_ice,dist_ice_edge_km
0,1999-12,1999,12,-69.239,75.05,-69.260,75.1,summer,weaning,1990,...,Antarctic,0,89.812088,504.080212,140.255558,777.86664,NaN,-1.506463,0.000000,724.237926
1,1999-12,1999,12,-69.239,75.85,-69.260,75.9,summer,weaning,1990,...,Antarctic,0,89.953110,506.158223,143.956368,503.00000,0.955706,-1.604321,0.535714,708.715428
2,1999-12,1999,12,-69.239,76.05,-69.260,76.1,summer,weaning,1990,...,Antarctic,0,89.948738,507.046161,145.996809,522.90000,0.945087,-1.573719,0.547619,705.058053
3,1999-12,1999,12,-69.239,77.25,-69.260,77.3,summer,weaning,1990,...,Antarctic,0,NaN,509.340574,166.208399,NaN,NaN,NaN,0.000000,685.100828
4,1999-12,1999,12,-69.155,73.95,-69.134,74.0,summer,weaning,1990,...,Antarctic,0,NaN,494.994640,130.924977,599.81740,NaN,-1.313767,0.000000,740.003921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35561,1999-12,1999,12,-59.442,69.25,-59.468,69.3,summer,weaning,1990,...,Antarctic,0,89.832825,556.221299,688.617402,4624.49700,0.000000,-0.626995,0.000000,289.197201
35562,1999-12,1999,12,-59.442,71.95,-59.468,72.0,summer,weaning,1990,...,Antarctic,0,89.652817,536.647811,699.380306,4429.30800,0.000000,-0.988253,0.035714,276.617004
35563,1987-11,1987,11,-59.340,70.85,-59.366,70.9,autumn,weaning,1980,...,Antarctic,0,89.979530,551.535259,703.061410,4767.58350,0.044812,-1.306596,0.000000,28.266072
35564,1998-11,1998,11,-59.340,71.55,-59.366,71.6,autumn,weaning,1990,...,Antarctic,0,89.929642,548.570655,707.329724,4608.48630,0.038312,-1.382793,0.035714,32.938866


## Saving data frame to disk

In [167]:
#Ensure output folder exists
os.makedirs(varDict['base_out'], exist_ok = True)

#Create file path where data will be saved
file_out = os.path.join(varDict['base_out'], 'unique_background_20x_obs_all_env.csv')

#Saving as csv file
crabeaters.to_csv(file_out, index = False)